In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
##############################################################################
# 1) INSTALL DEPENDENCIES
##############################################################################
# In a fresh Colab / environment, run:
!pip install faiss-cpu sentence-transformers transformers accelerate scikit-learn matplotlib seaborn
!pip install --upgrade git+https://github.com/huggingface/transformers.git
!pip install torch bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 20.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-qys8vuo4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-qys8vuo4
  Resolved https://github.com/huggingface/transformers.git to commit 5e2183f344911aa82aba0b83778a4f196cff378e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10772874 sha256=1feecc006155d3e9bb4d17c7b46a13879f5088242c56a04258fd3c3e8c75fb3c
  Stored in directory: /tmp/pip-ephem-wheel-cache-m5o_pm0_/wheels/e7/9c/5b/e1a9

In [3]:
"""
Business Plan Validation with DeepSeek-R1
-----------------------------------------
Use the advanced DeepSeek-R1 model to IDEATE, VALIDATE, and CREATE
business plans. Demonstrates:
1) Loading DeepSeek-R1 from Hugging Face
2) Generating plan feedback (interactive)
3) A small RAG flow with FAISS to incorporate "business best practice" docs
"""
import torch
import textwrap
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)
from transformers import AutoTokenizer, AutoModelForCausalLM

# For the RAG portion (FAISS-based retrieval), we need faiss and sentence_transformers
try:
    import faiss
    from sentence_transformers import SentenceTransformer
    FAISS_AVAILABLE = True
except ImportError:
    FAISS_AVAILABLE = False
    print("FAISS or sentence_transformers not installed; RAG demo will be skipped.")

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
E0000 00:00:1740048423.643178      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:230


In [4]:
def load_deepseek_model():
    """
    Loads deepseek-7b-chat (works on T4/V100 GPUs or even CPU with enough RAM).
    """
    model_name = "deepseek-ai/deepseek-llm-7b-chat"  # Compatible alternative
    print(f"Loading model: {model_name} ...")

    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,  # Works with older GPUs
        device_map="auto",
        trust_remote_code=True,
        #load_in_4bit=True,  # 4-bit quantization
       # bnb_4bit_use_double_quant=True,
        #bnb_4bit_quant_type="nf4",
    )
    print("DeepSeek-7B loaded successfully!")
    return tokenizer, model

tokenizer, model = load_deepseek_model()

Loading model: deepseek-ai/deepseek-llm-7b-chat ...


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.48it/s]


DeepSeek-7B loaded successfully!


In [5]:
def generate_plan_feedback(
    prompt: str,
    max_new_tokens: int = 400,
    temperature: float = 0.6,
    top_p: float = 0.95
) -> str:
    """
    Generates text from DeepSeek-R1. Tailored for business plan validation.

    :param prompt: Business plan query or section
    :param max_new_tokens: Maximum tokens in the output
    :param temperature: Controls creativity
    :param top_p: Nucleus sampling threshold
    :return: Model's generated response about the business plan
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()

In [6]:
def setup_business_docs_faiss():
    """
    Create a small set of business best-practice docs, index them with FAISS.
    Returns:
      doc_texts, doc_ids, index, embed_model
    """
    # Some short business or startup practice docs
    doc_texts = [
        """Doc #1 - Business Plan Basics:
        - Clearly define value proposition
        - Identify target market & competition
        - Create financial projections & funding strategy
        - Validate product-market fit through customer research
        - Develop go-to-market strategy
        """,
        """Doc #2 - Startup Growth & Funding:
        - Secure initial funding or bootstrap
        - Explore different revenue models (SaaS, subscription, ads)
        - Keep track of cash flow & runway
        - Seek investors if you have proven traction
        """,
        """Doc #3 - Scaling & Operations:
        - Build a strong team & delegation system
        - Optimize supply chain or delivery process
        - Automate repetitive tasks to reduce costs
        - Stay agile & pivot if data suggests
        """
    ]
    doc_ids = [f"doc_{i}" for i in range(len(doc_texts))]
     # Convert to embeddings using sentence transformers
    sbert = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    embeddings = sbert.encode(doc_texts, convert_to_numpy=True)
    dim = embeddings.shape[1]

    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    return doc_texts, doc_ids, index, sbert

def retrieve_docs_faiss(query: str, sbert, doc_texts, doc_ids, index, top_k=1):
    """
    Retrieves top_k documents from FAISS index for the user's query.
    """
    query_vec = sbert.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vec, top_k)

    retrieved = []
    for rank, idx in enumerate(indices[0]):
        retrieved.append({
            "id": doc_ids[idx],
            "text": doc_texts[idx],
            "distance": distances[0][rank]
        })
    return retrieved

def generate_rag_response(query: str, retrieved_context: str) -> str:
    """
    Combine user query + retrieved doc text into a final prompt for plan validation.
    """
    # We add a note encouraging the model to reason step by step if you want more thorough feedback
    prompt = f"""
You are an advanced business plan validator. Read the context, then address the user's question.

Context:
{retrieved_context}

User Query: {query}

Business Plan Feedback:
"""
    return generate_plan_feedback(prompt)


In [7]:
def interactive_business_plan_demo():
    """
    1) Takes user input about business plan sections or queries.
    2) Returns advanced LLM-based feedback from DeepSeek-R1.
    3) Optionally uses RAG with FAISS if user wants to incorporate best-practice docs.
    """
    print("\n🔹🔹 Validate Your Business Plan with DeepSeek-R1 🔹🔹")
    print("Enter a question or statement about your business plan to get AI feedback.")
    print("Type 'exit' to quit.\n")

    # Optional: set up FAISS-based docs
    if FAISS_AVAILABLE:
        doc_texts, doc_ids, index, sbert = setup_business_docs_faiss()
        print("RAG is enabled. We'll retrieve relevant best-practice docs to enrich feedback.\n")
    else:
        print("FAISS not installed => No document retrieval. We'll do direct LLM feedback.\n")
        doc_texts = doc_ids = index = sbert = None
    while True:
        user_input = input("Business Plan Input: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("\nExiting. Goodbye!\n")
            break

        if FAISS_AVAILABLE:
            # Retrieve doc
            top_docs = retrieve_docs_faiss(user_input, sbert, doc_texts, doc_ids, index, top_k=1)
            context_block = "\n".join([doc['text'] for doc in top_docs])
            response = generate_rag_response(user_input, context_block)
            print("\n--- Retrieved Context ---\n")
            print(textwrap.fill(context_block, width=100))
            print("\n--- AI Feedback ---\n")
            print(textwrap.fill(response, width=100))
            print("\n" + "="*80)
        else:
            # Simple direct call
            response = generate_plan_feedback(user_input)
            print("\nAI Feedback:\n" + textwrap.fill(response, width=100))
            print("\n" + "="*80)

In [8]:
!pip install textblob
!pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 9.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.9 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.5 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [9]:

from textblob import TextBlob  # For sentiment analysis

# Performance evaluation
def evaluate_performance(y_true, y_pred):
    """Calculate and display performance metrics."""
    if len(set(y_true)) == 1 or len(set(y_pred)) == 1:
        print("⚠️ Warning: Only one class present in y_true or y_pred. Metrics may be unreliable.")
        return

    print("\n📊 Performance Metrics:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.2f}")
    print(f"Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.2f}")
    print(f"Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.2f}")
    print(f"F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.2f}")
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, zero_division=0))

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()


In [11]:

# Interactive demo
def interactive_business_plan_demo():
    """Interactive session to validate business plans."""
    print("\n🔹🔹 Validate Your Business Plan with DeepSeek-R1 🔹🔹")
    print("Enter a question or statement about your business plan to get AI feedback.")
    print("Type 'exit' to quit.\n")

    # Optional: set up FAISS-based docs
    if FAISS_AVAILABLE:
        doc_texts, doc_ids, index, sbert = setup_business_docs_faiss()
        print("RAG is enabled. We'll retrieve relevant best-practice docs to enrich feedback.\n")
    else:
        print("FAISS not installed => No document retrieval. We'll do direct LLM feedback.\n")
        doc_texts = doc_ids = index = sbert = None

    # For performance evaluation
    y_true = []  # Ground truth labels (e.g., manually annotated)
    y_pred = []  # Predicted labels (e.g., based on feedback quality)

    while True:
        user_input = input("Business Plan Input: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("\nExiting. Goodbye!\n")
            break

        if FAISS_AVAILABLE:
            # Retrieve doc
            top_docs = retrieve_docs_faiss(user_input, sbert, doc_texts, doc_ids, index, top_k=1)
            context_block = "\n".join([doc['text'] for doc in top_docs])
            response = generate_rag_response(user_input, context_block)
            print("\n--- Retrieved Context ---\n")
            print(textwrap.fill(context_block, width=100))
            print("\n--- AI Feedback ---\n")
            print(textwrap.fill(response, width=100))
            print("\n" + "=" * 80)
        else:
            # Simple direct call
            response = generate_plan_feedback(user_input)
            print("\nAI Feedback:\n" + textwrap.fill(response, width=100))
            print("\n" + "=" * 80)

        # Simulate performance evaluation (replace with actual ground truth)
        y_true.append(1)  # Assume feedback is correct
        y_pred.append(1 if "good" in response.lower() else 0)  # Simple heuristic for prediction

    # Evaluate performance at the end
    evaluate_performance(y_true, y_pred)

if __name__ == "__main__":
    # Run an interactive session to validate business plans
    interactive_business_plan_demo()


🔹🔹 Validate Your Business Plan with DeepSeek-R1 🔹🔹
Enter a question or statement about your business plan to get AI feedback.
Type 'exit' to quit.

RAG is enabled. We'll retrieve relevant best-practice docs to enrich feedback.



Business Plan Input:  My plan has no clear market strategy.



--- Retrieved Context ---

Doc #1 - Business Plan Basics:         - Clearly define value proposition         - Identify target
market & competition         - Create financial projections & funding strategy         - Validate
product-market fit through customer research         - Develop go-to-market strategy

--- AI Feedback ---

You are an advanced business plan validator. Read the context, then address the user's question.
Context: Doc #1 - Business Plan Basics:         - Clearly define value proposition         -
Identify target market & competition         - Create financial projections & funding strategy
- Validate product-market fit through customer research         - Develop go-to-market strategy
User Query: My plan has no clear market strategy.  Business Plan Feedback:  Considering the context
provided, it appears that your business plan may be lacking a clear market strategy. A solid market
strategy is crucial for any business plan, as it outlines how you plan to reach and ap

Business Plan Input:  What red flags should I remove from my funding proposal?



--- Retrieved Context ---

Doc #2 - Startup Growth & Funding:         - Secure initial funding or bootstrap         - Explore
different revenue models (SaaS, subscription, ads)         - Keep track of cash flow & runway
- Seek investors if you have proven traction

--- AI Feedback ---

You are an advanced business plan validator. Read the context, then address the user's question.
Context: Doc #2 - Startup Growth & Funding:         - Secure initial funding or bootstrap         -
Explore different revenue models (SaaS, subscription, ads)         - Keep track of cash flow &
runway         - Seek investors if you have proven traction           User Query: What red flags
should I remove from my funding proposal?  Business Plan Feedback: To improve your funding proposal
and remove any potential red flags, consider the following recommendations:  1. Address key
concerns: Ensure your proposal addresses the main concerns of potential investors. This includes a
clear and concise description of

...


--- Retrieved Context ---

Doc #1 - Business Plan Basics:         - Clearly define value proposition         - Identify target
market & competition         - Create financial projections & funding strategy         - Validate
product-market fit through customer research         - Develop go-to-market strategy

--- AI Feedback ---

You are an advanced business plan validator. Read the context, then address the user's question.
Context: Doc #1 - Business Plan Basics:         - Clearly define value proposition         -
Identify target market & competition         - Create financial projections & funding strategy
- Validate product-market fit through customer research         - Develop go-to-market strategy
User Query: What strategies can I use to better define my competitive advantage?  Business Plan
Feedback: To better define your competitive advantage, consider the following strategies:  1.
Differentiation: Identify what sets your product or service apart from the competition. This can

Business Plan Input:  What are the key elements missing in my business plan?



--- Retrieved Context ---

Doc #1 - Business Plan Basics:         - Clearly define value proposition         - Identify target
market & competition         - Create financial projections & funding strategy         - Validate
product-market fit through customer research         - Develop go-to-market strategy

--- AI Feedback ---

You are an advanced business plan validator. Read the context, then address the user's question.
Context: Doc #1 - Business Plan Basics:         - Clearly define value proposition         -
Identify target market & competition         - Create financial projections & funding strategy
- Validate product-market fit through customer research         - Develop go-to-market strategy
User Query: What are the key elements missing in my business plan?  Business Plan Feedback: Based on
the context provided, your business plan appears to be missing a few key elements:  1. Value
Proposition: It is essential to clearly define the unique value your product or service offe

Business Plan Input:  exit



Exiting. Goodbye!

⚠️ Warning: Only one class present in y_true or y_pred. Metrics may be unreliable.
